# How People Decide what they want to do
Directed graph approach

Generally people want to do a number of different things. For this I'm going to create a schema for this in a graph language that allows me to designate how much a `pop` desires to take a certain action. This will be used later when determining AI decisions. 
Note that this does not reflect on the player's actions. These are the automated actions of people. For example: a people that desire war, and aren't loayl to a faction may rebel. 

**Note** this notebook actualy builds the desires into the graph, overwriting existing ontology. Nodes that do this are commented out.



In [4]:
import sys
import numpy as np
import pandas as pd
import altair as alt
sys.path.append('..')
import helpers.dbquery as db
import helpers.functions as f
import yaml, ssl, asyncio
import nb_black

ssl._create_default_https_context = ssl._create_unverified_context
asyncio.set_event_loop_policy(asyncio.WindowsSelectorEventLoopPolicy())
import nest_asyncio
# this is required for running in a Jupyter Notebook. 
nest_asyncio.apply()

In [5]:
local_user = "BillmanLocal2"
res = db.run_query(f"g.V().hasLabel('pop').has('username','{local_user}').valueMap()")
pops = [db.clean_node(n) for n in res]
pops[0]

{'conformity': 0.306,
 'literacy': 0.925,
 'aggression': 0.502,
 'constitution': 0.401,
 'objid': '5720114744401',
 'faction_no': 2,
 'name': 'Bachna Ca',
 'isInFaction': '1987887122089',
 'industry': 0.4515,
 'wealth': 0.6882,
 'faction_loyalty': 0.4665,
 'username': 'BillmanLocal2',
 'objtype': 'pop',
 'id': '5720114744401'}

Each population wants to do everything to a degree, the amount of desire to do that thing is expressed by the edge weight. 
* Attack a population
* Focus on improving literacy
* Focus on improving industry

# Desires as Objects

## Desire with targets.
Both factions and pops can have desire. Action is guided by desire based on the `max(desire.weight)`.
`desire` is an edge, the type of that desire is a property of that edge, and the edge weight is the amount of desire. The target (`node2`) is the recipient. 

Examples:
* faction wants trade with faction
* pop wants war with another pop
* pop wants faction to go to war with faction

## Desire without targets.

Desires without targets must link to an objective. That objective can be it's own node.

### This next cell will upload new desires from `desires.yaml`, which can be edited at any time. 

desires.yaml can be used to configure how people act as a default. This can be helpful in adjusting the default settings in your universe. 


In [6]:
# # Drop the items, if they exist. 
# db.run_query("g.V().hasLabel('objective').has('username','notebook').drop()")
# objectives_yaml = yaml.safe_load(open("desires.yaml"))['objectives']
# data = {"nodes":objectives_yaml,'edges':[]}
# # Then Create the nodes and add them to the DB
# db.upload_data(data,verbose=False)
# After creating the nodes, pulling them into the notebook for reference
# I won't re-create them here, unless I've updated the file. 
res = db.run_query("g.V().hasLabel('objective').valueMap()")
objectives = [db.clean_node(n) for n in res]
pd.DataFrame(objectives)


,type,weight,leadingAttribute,comment,username,objtype,objid,id
0,industry,0.5,wealth,"build factories, skyscrapers, infrastructure",notebook,objective,2008795542230,2008795542230
1,expansion,0.5,industry,"to increase population, eventually becoming a ...",notebook,objective,3832546566671,3832546566671
2,war,0.4,aggression,the general amount that the society wants to f...,notebook,objective,9111777612067,9111777612067
3,wealth,0.5,literacy,"increasing luxury, entertainiment and amenities",notebook,objective,9171875985828,9171875985828
4,science,0.5,literacy,"building schools, education systems, informati...",notebook,objective,7056154012146,7056154012146


# population wants to improve industry

populations want to improve industry when:
* they are not wealty
* they are at war




In [7]:
# Marginal return on base attribute
n = 2
ind_df = pd.DataFrame(np.sort([float(p['wealth']) for p in pops]),columns=['wealth'])
ind_df['base'] = range(len(ind_df))
ind_df['desires_industry'] = ind_df['wealth'].apply(lambda x: ((x+1)**(1-n) - 1)/(1-n))
ind_df['desire_base'] = ind_df['base'].apply(lambda x: ((x+1)**(1-n) - 1)/(1-n))
alt.Chart(ind_df).mark_line().encode(x='base',y='desire_base').properties(title="Desire relative to the base attribute")

alt.Chart(...)

In [8]:
alt.Chart(ind_df).mark_line().encode(x='wealth:N',y='desires_industry').properties(title="Desires wealth industry relative to industry")

alt.Chart(...)

## feeding that desire to the populations

Per above, the initial desire to do a thing is dependant on the `leading attribute`. This makes it easy to adjust the population desires in the `desires.yaml`.

In [9]:
def get_desire(x):
       return np.round(((float(x)+1)**(1-n) - 1)/(1-n),3)

popno = 0
objectiveno = 0

print(pops[popno])
print(objectives[objectiveno])

{'conformity': 0.306, 'literacy': 0.925, 'aggression': 0.502, 'constitution': 0.401, 'objid': '5720114744401', 'faction_no': 2, 'name': 'Bachna Ca', 'isInFaction': '1987887122089', 'industry': 0.4515, 'wealth': 0.6882, 'faction_loyalty': 0.4665, 'username': 'BillmanLocal2', 'objtype': 'pop', 'id': '5720114744401'}
{'type': 'industry', 'weight': '0.5', 'leadingAttribute': 'wealth', 'comment': 'build factories, skyscrapers, infrastructure', 'username': 'notebook', 'objtype': 'objective', 'objid': '2008795542230', 'id': '2008795542230'}


In [10]:
get_desire(pops[popno][objectives[objectiveno]['leadingAttribute']])

0.408

Create a desire edge for each desire, for each population.

In [11]:
edges = []
for p in pops:
    for o in objectives:
        edge = {'label':'desires',
                'node1':p['objid'],
                'node2':o['objid'],
                'desire':o['type'],
                'weight':get_desire(p[o['leadingAttribute']])}
        edges.append(edge)

In [12]:
o

{'type': 'science',
 'weight': '0.5',
 'leadingAttribute': 'literacy',
 'comment': 'building schools, education systems, information infrastructure',
 'username': 'notebook',
 'objtype': 'objective',
 'objid': '7056154012146',
 'id': '7056154012146'}

In [13]:
pd.DataFrame(edges)

,label,node1,node2,desire,weight
0,desires,5720114744401,2008795542230,industry,0.408
1,desires,5720114744401,3832546566671,expansion,0.311
2,desires,5720114744401,9111777612067,war,0.334
3,desires,5720114744401,9171875985828,wealth,0.481
4,desires,5720114744401,7056154012146,science,0.481
5,desires,1590902300324,2008795542230,industry,0.398
6,desires,1590902300324,3832546566671,expansion,0.281
7,desires,1590902300324,9111777612067,war,0.195
8,desires,1590902300324,9171875985828,wealth,0.483
9,desires,1590902300324,7056154012146,science,0.483


In [14]:
db.create_edge(edges[0], local_user)

"g.V().has('objid','5720114744401').addE('desires').property('username','BillmanLocal2').property('desire','industry').property('weight',0.408).to(g.V().has('objid','2008795542230'))"

I'm not actually going to upload the edge, as this function is in production

In [15]:
# db.upload_data({'nodes':[],'edges':edges},verbose=False)

# Choosing based on the strongest desire

Here is how you take one pop, and calculate what it desires. 

In [16]:
query = "g.V().hasLabel('objective').valueMap()"
db.qtodf(query)

,type,weight,leadingAttribute,comment,username,objtype,objid,id
0,industry,0.5,wealth,"build factories, skyscrapers, infrastructure",notebook,objective,2008795542230,2008795542230
1,expansion,0.5,industry,"to increase population, eventually becoming a ...",notebook,objective,3832546566671,3832546566671
2,war,0.4,aggression,the general amount that the society wants to f...,notebook,objective,9111777612067,9111777612067
3,wealth,0.5,literacy,"increasing luxury, entertainiment and amenities",notebook,objective,9171875985828,9171875985828
4,science,0.5,literacy,"building schools, education systems, informati...",notebook,objective,7056154012146,7056154012146


In [17]:
db.qtodf(f"g.V().hasLabel('pop').has('username','{local_user}').limit(2).valueMap()")

,conformity,literacy,aggression,constitution,objid,faction_no,name,isInFaction,industry,wealth,faction_loyalty,username,objtype,id
0,0.306,0.925,0.502,0.401,5720114744401,2,Bachna Ca,1987887122089,0.4515,0.6882,0.4665,BillmanLocal2,pop,5720114744401
1,0.442,0.933,0.242,0.541,1590902300324,1,Unty De,2476940103283,0.3915,0.6622,0.4605,BillmanLocal2,pop,1590902300324


Arbitrarily grabbing a pop id: `9033676237817`

In [18]:
popid1 = "5720114744401"
popid2 = "1590902300324"

I'll likely revisit this query, but it moves the project along for now. This returns the desire for each objective. This can be expanded as it grows. 

In [25]:
res = db.run_query(f"g.V().has('objid','{popid1}').outE('desires').inV().hasLabel('objective').dedup().path().by('name').by('weight').by('type')")
res

[{'labels': [[], [], []], 'objects': ['Bachna Ca', 0.334, 'war']},
 {'labels': [[], [], []], 'objects': ['Bachna Ca', 0.481, 'wealth']},
 {'labels': [[], [], []], 'objects': ['Bachna Ca', 0.311, 'expansion']},
 {'labels': [[], [], []], 'objects': ['Bachna Ca', 0.408, 'industry']},
 {'labels': [[], [], []], 'objects': ['Bachna Ca', 0.481, 'science']}]

In [31]:
pd.DataFrame([i['objects'] for i in res], columns=['pop','desire','objective']).sort_values('desire',ascending=False).reset_index(drop=True)

,pop,desire,objective
0,Bachna Ca,0.481,wealth
1,Bachna Ca,0.481,science
2,Bachna Ca,0.408,industry
3,Bachna Ca,0.334,war
4,Bachna Ca,0.311,expansion


Automated functions can take a people's greatest desire and place actions on thier behalf. 
